In [5]:
import numpy as np

def mod2(matrix):
    """Apply element-wise mod 2 to a matrix."""
    return np.mod(matrix, 2)


def reduce_to_echelon_form(A):
    """
    Reduce the matrix A to echelon form over GF(2) and return the row operations matrix (M),
    column operations matrix (N), and the rank (rnk).
    """
    rows, cols = A.shape
    A = np.copy(A) % 2  # Ensure entries are in GF(2)

    # Initialize row and column operations as identity matrices
    M = np.eye(rows, dtype=int)  # Row operations matrix
    N = np.eye(cols, dtype=int)  # Column operations matrix

    row = 0  # Start from the top row

    for col in range(cols):
        if row >= rows:
            break  # Stop if all rows are processed

        # Find a pivot row for the current column
        pivot = np.where(A[row:, col] == 1)[0]
        if len(pivot) == 0:
            continue  # No pivot in this column, move to the next column

        pivot_row = pivot[0] + row  # Adjust pivot index to absolute position

        # Swap the current row with the pivot row
        A[[row, pivot_row]] = A[[pivot_row, row]]
        M[[row, pivot_row]] = M[[pivot_row, row]]  # Track row swap in M

        # Eliminate all other ones in this column
        for r in range(rows):
            if r != row and A[r, col] == 1:
                A[r] = (A[r] + A[row]) % 2  # Row elimination using GF(2) addition
                M[r] = (M[r] + M[row]) % 2  # Track the row operation in M

        # Track column swap in N if row != col (to ensure identity structure)
        if col != row:
            N[:, [col, row]] = N[:, [row, col]]  # Track column swap in N

        row += 1  # Move to the next row

    # Compute the rank as the number of non-zero rows in the reduced form
    rnk = np.sum(np.any(A, axis=1))

    return A, M, N, rnk


def gf2matinv(matrix):
    """
        Compute the inverse of a matrix in GF(2) using Gaussian elimination.

        Parameters:
        matrix (numpy.ndarray): A square matrix to invert in GF(2).

        Returns:
        numpy.ndarray: The inverse of the matrix in GF(2).

        Raises:
        ValueError: If the matrix is singular (i.e., not invertible).
    """
    matrix = np.array(matrix, dtype=np.int8) % 2  # Ensure GF(2) operations
    m = len(matrix)
    aug_matrix = np.concatenate((matrix, np.eye(m, dtype=np.int8)), axis=1)  # Augment with identity matrix

    # Gaussian elimination for GF(2)
    for col in range(m):
        for row in range(col, m):
            if aug_matrix[row, col]:
                if row != col:
                    aug_matrix[[col, row]] = aug_matrix[[row, col]]  # Swap rows
                break
        else:
            raise ValueError("Matrix is singular over GF(2)")

        for i in range(m):
            if i != col and aug_matrix[i, col]:
                aug_matrix[i] = (aug_matrix[i] + aug_matrix[col]) % 2  # Row operation

    return aug_matrix[:, m:]  # Return the inverse portion


In [6]:
def symp_mat_decompose(F):
    """Decompose a symplectic matrix into elementary symplectic transformations."""
    m = F.shape[0] // 2
    I = np.eye(m, dtype=int)
    O = np.zeros((m, m), dtype=int)

    def U(k, m):
        """Creates a block diagonal matrix with an identity of size k 
        and a zero matrix of size (m-k)."""
        return np.block([
            [np.eye(k), np.zeros((k, m - k))],
            [np.zeros((m - k, k)), np.zeros((m - k, m - k))]
        ])
    def L(k, m):
        """Create a block diagonal matrix with zeros and I_k."""
        if k > m:
            raise ValueError("k should be less than or equal to m.")
        # Create the block matrix using np.block
        return np.block([
        [np.zeros((m - k, m - k), dtype=int), np.zeros((m - k, k), dtype=int)],
        [np.zeros((k, m - k), dtype=int), np.eye(k, dtype=int)]
        ])
    
    Omega = np.block([[O, I], [I, O]])  # Transversal Hadamard
    Elem1 = lambda Q: np.block([[Q, np.zeros_like(Q)], [np.zeros_like(Q), gf2matinv(Q)]])
    Elem2 = lambda R: np.block([[I, R], [O, I]])
    def Elem3(k, m):
        """Creates the Elem3 matrix using L and U."""
        upper = np.block([L(m - k, m), U(k, m)])
        lower = np.block([U(k, m), L(m - k, m)])
        return np.block([
            [upper],
            [lower]
        ])
    
    A = F[:m, :m]
    B = F[:m, m:]
    C = F[m:, :m]
    D = F[m:, m:]

    if ((np.all(A == I) and np.all(C == O) and np.all(D == I)) or
        (np.all(B == O) and np.all(C == O)) or
        np.all(F == Omega)):
        return [F]

    # Step 1
    _, M_A, N_A, k = reduce_to_echelon_form(A)
   
    Qleft1 = Elem1(M_A)
    Qright = Elem1(N_A)
    Fcp = mod2(Qleft1 @ F @ Qright)
    
    if k == m:
        Rright = Elem2(Fcp[:m, m:])
        Fcp = mod2(Fcp @ Rright)
        R = Fcp[m:, :m]
        return [gf2matinv(Qleft1), Omega, Elem2(R), Omega, gf2matinv(Rright), gf2matinv(Qright)]
    
    # Step 2
    Bmk = Fcp[k:m, m + k:]
    _, M_Bmk1, _, _ = reduce_to_echelon_form(Bmk)
    
    M_Bmk = np.block([[np.eye(k, dtype=int), np.zeros((k, m - k), dtype=int)],
                      [np.zeros((m - k, k), dtype=int), M_Bmk1]])
   
    Qleft2 = Elem1(M_Bmk)
   
    Fcp = mod2(Qleft2 @ Fcp)
   
    # Step 3
    E = Fcp[:k, m + k:]
    M_E = np.block([[np.eye(k, dtype=int), E], [np.zeros((m - k, k), dtype=int), np.eye(m - k, dtype=int)]])
    Qleft3 = Elem1(M_E)
    Fcp = mod2(Qleft3 @ Fcp)
    

    # Step 4
    S = Fcp[:k, :k]
    Rright = Elem2(np.block([[S, np.zeros((k, m - k), dtype=int)], [np.zeros((m - k, m), dtype=int)]]))
    Fcp = mod2(Fcp @ Rright)
    
    # Step 5
    
    
    Fright = mod2(Omega @ Elem3(k,m))
    Fcp = mod2(Fcp @ Fright)
    R = Fcp[m:, :m]

    Q = mod2(Qleft3 @ Qleft2 @ Qleft1)
    return [gf2matinv(Q), Omega, Elem2(R), Elem3(k,m), gf2matinv(Rright), gf2matinv(Qright)]


In [7]:
U = np.array([[0., 0., 1., 0.],
        [0., 0., 1., 1.],
        [1., 1., 1., 0.],
        [0., 1., 1., 1.]])
V = symp_mat_decompose(U)
V

[array([[1, 0, 0, 0],
        [1, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 1]], dtype=int8),
 array([[0, 0, 1, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 1, 0, 0]]),
 array([[1., 0., 1., 0.],
        [0., 1., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 array([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]], dtype=int8),
 array([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]], dtype=int8)]

In [8]:
import numpy as np

def gf2lu(A):
    """
    Perform LU decomposition of A over GF(2) such that P @ A = L @ U,
    where P is a permutation matrix, L is a lower triangular matrix, 
    and U is an upper triangular matrix in row-echelon form.
    """
    m = A.shape[0]
    U = A.copy()
    L = np.eye(m, dtype=int)
    P = np.eye(m, dtype=int)

    for k in range(m - 1):
        # Find the pivot row
        pivot = np.where(U[k:m, k] == 1)[0]
        if len(pivot) == 0:
            continue
        i = pivot[0] + k  # Adjust to absolute row index

        # Swap rows in U, L, and P
        U[[k, i], k:] = U[[i, k], k:]
        L[[k, i], :k] = L[[i, k], :k]
        P[[k, i], :] = P[[i, k], :]

        # Perform row elimination
        for j in range(k + 1, m):
            L[j, k] = U[j, k]
            U[j, k:] = (U[j, k:] - L[j, k] * U[k, k:]) % 2

    return L, U, P

def find_circuit(F):
    """
    Find a quantum circuit for the given symplectic transformation F 
    using Trung Can's algorithm to decompose F into elementary gates.
    """
    m = F.shape[0] // 2
    I = np.eye(m, dtype=int)
    O = np.zeros((m, m), dtype=int)
    Omega = np.block([[O, I], [I, O]])

    # Validate symplectic matrix
    if not np.all(mod2(F @ Omega @ F.T) == Omega):
        print('\nInvalid symplectic matrix!')
        return []

    if np.all(F == np.eye(2 * m, dtype=int)):
        return []

    # Decompose F into elementary symplectic transformations
    Decomp = symp_mat_decompose(F)
    circuit = []

    for matrix in Decomp:
        if np.all(matrix == np.eye(2 * m, dtype=int)):
            continue
        elif np.all(matrix == Omega):
            circuit.append(('H', list(range(1, m + 1))))
            continue

        A = matrix[:m, :m]
        B = matrix[:m, m:]
        C = matrix[m:, :m]
        D = matrix[m:, m:]

        if np.all(A == I) and np.all(C == O) and np.all(D == I):
            # CZs and Phase gates
            P_ind = np.where(np.diag(B) == 1)[0].tolist()
            if P_ind:
                circuit.append(('P', P_ind))

            B = np.triu(mod2(B + np.diag(np.diag(B))))
            for j in range(m):
                CZ_ind = np.where(B[j] == 1)[0]
                for k in CZ_ind:
                    circuit.append(('CZ', [j + 1, k + 1]))

        elif np.all(B == O) and np.all(C == O):
            # CNOTs and Permutations using LU decomposition over GF(2)
            L, U, P = gf2lu(A)

            if not np.all(P == I):
                circuit.append(('Permute', (np.arange(1, m + 1) @ P.T).tolist()))

            for j in range(m):
                inds = np.setdiff1d(np.where(L[j] == 1)[0], [j])
                for k in inds:
                    circuit.append(('CNOT', [j + 1, k + 1]))

            for j in range(m - 1, -1, -1):
                inds = np.setdiff1d(np.where(U[j] == 1)[0], [j])
                for k in inds:
                    circuit.append(('CNOT', [j + 1, k + 1]))

        else:
            # Partial Hadamards
            k = m - np.sum(np.diag(A))
            Uk = np.block([[np.eye(k, dtype=int), np.zeros((k, m - k), dtype=int)]])
            Lmk = np.block([[np.zeros((m - k, k), dtype=int), np.eye(m - k, dtype=int)]])

            if np.all(A == Lmk) and np.all(B == Uk) and np.all(C == Uk) and np.all(D == Lmk):
                circuit.append(('H', list(range(1, k + 1))))
            else:
                print('\nUnknown elementary symplectic form!')
                return []

    return circuit

def mod2(matrix):
    """Perform element-wise mod 2 operation."""
    return np.mod(matrix, 2)

# Assuming you have defined symp_mat_decompose elsewhere in your code.


In [9]:
F = np.array([
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
], dtype=int)

circuit = find_circuit(F)
print("Circuit:", circuit)

Circuit: [('Permute', [2, 1])]
